In [1]:
!unzip /content/drive/MyDrive/data.zip

Archive:  /content/drive/MyDrive/data.zip
   creating: data/
  inflating: data/frame_00146_center.jpg  
  inflating: data/frame_00057_left.jpg  
  inflating: data/frame_00498_center.jpg  
  inflating: data/frame_00188_center.jpg  
  inflating: data/frame_00290_center.jpg  
  inflating: data/frame_00492_left.jpg  
  inflating: data/frame_00218_center.jpg  
  inflating: data/frame_00276_left.jpg  
  inflating: data/frame_00374_center.jpg  
  inflating: data/frame_00140_center.jpg  
  inflating: data/frame_00263_center.jpg  
  inflating: data/frame_00052_right.jpg  
  inflating: data/frame_00326_center.jpg  
  inflating: data/frame_00069_left.jpg  
  inflating: data/frame_00005_right.jpg  
  inflating: data/frame_00055_left.jpg  
  inflating: data/frame_00318_right.jpg  
  inflating: data/frame_00216_left.jpg  
  inflating: data/frame_00212_center.jpg  
  inflating: data/frame_00376_center.jpg  
  inflating: data/frame_00556_center.jpg  
  inflating: data/frame_00406_center.jpg  
  inflat

In [3]:
# === 1. 라이브러리 임포트 ===
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import os

# === 2. 하이퍼파라미터 설정 ===
BATCH_SIZE = 32
EPOCHS = 20
LEARNING_RATE = 1e-3
IMAGE_SIZE = (160, 120)  # 작은 해상도로 resize

# === 3. 데이터셋 정의 ===
class DrivingDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_path = os.path.join(self.img_dir, row["frame"])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        angle = torch.tensor([row["steering_angle"] / 180.0], dtype=torch.float32)  # 정규화
        return image, angle

# === 4. 이미지 전처리 ===
transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # -1 ~ 1 범위
])

# === 5. 데이터 불러오기 ===
dataset = DrivingDataset(csv_file='/content/data/drive_log.csv', img_dir='/content/data', transform=transform)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# === 6. 회귀 CNN 모델 정의 ===
class RegressionCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 16, 5, stride=2), nn.ReLU(),    # [3,160,120] -> [16, 78, 58]
            nn.Conv2d(16, 32, 5, stride=2), nn.ReLU(),   # -> [32, 37, 27]
            nn.Conv2d(32, 64, 5, stride=2), nn.ReLU()    # -> [64, 17, 12]
        )
        self.flatten = nn.Flatten()
        self.fc = nn.Sequential(
            nn.Linear(64 * 17 * 12, 100), nn.ReLU(),
            nn.Linear(100, 1)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x


# === 7. 학습 설정 ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RegressionCNN().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# === 8. 학습 루프 ===
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0.0
    for images, angles in dataloader:
        images, angles = images.to(device), angles.to(device)
        outputs = model(images)
        loss = criterion(outputs, angles)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    print(f"Epoch [{epoch+1}/{EPOCHS}] Loss: {epoch_loss/len(dataloader):.4f}")

# === 9. 모델 저장 ===
torch.save(model.state_dict(), "model_regression.pth")
print("✅ 모델 저장 완료: model_regression.pth")


Epoch [1/20] Loss: 0.0726
Epoch [2/20] Loss: 0.0138
Epoch [3/20] Loss: 0.0083
Epoch [4/20] Loss: 0.0070
Epoch [5/20] Loss: 0.0063
Epoch [6/20] Loss: 0.0058
Epoch [7/20] Loss: 0.0059
Epoch [8/20] Loss: 0.0045
Epoch [9/20] Loss: 0.0042
Epoch [10/20] Loss: 0.0051
Epoch [11/20] Loss: 0.0040
Epoch [12/20] Loss: 0.0035
Epoch [13/20] Loss: 0.0030
Epoch [14/20] Loss: 0.0025
Epoch [15/20] Loss: 0.0024
Epoch [16/20] Loss: 0.0022
Epoch [17/20] Loss: 0.0019
Epoch [18/20] Loss: 0.0022
Epoch [19/20] Loss: 0.0017
Epoch [20/20] Loss: 0.0016
✅ 모델 저장 완료: model_regression.pth
